
### Examining racial discrimination in the US job market

#### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

#### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes.

#### Exercise
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Discuss statistical significance.

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

****

In [38]:
import pandas as pd
import numpy as np
from scipy import stats

In [39]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [4]:
# number of callbacks for balck-sounding names
sum(data[data.race=='b'].call)

157.0

In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [6]:
data.race.head()

0    w
1    w
2    b
3    b
4    w
Name: race, dtype: object

In [7]:
data.call.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: call, dtype: float32

In [8]:
len(data)

4870

In [15]:
success = data[data.call==1]
len(success)

392

In [16]:
len(success)/len(data)

0.08049281314168377

In [18]:
black = data[data.race=='b']
len(black)

2435

In [20]:
black_success =  sum(black.call)
black_success

157.0

In [21]:
white = data[data.race=='w']
len(white)

2435

In [22]:
white_success =  sum(white.call)
white_success 

235.0

In [26]:
white_success/len(white), black_success/len(black) #white and black success rates

(0.096509240246406572, 0.064476386036960986)

In [29]:
white.call.head()

0    0.0
1    0.0
4    0.0
5    0.0
6    0.0
Name: call, dtype: float32

In [36]:
white.call.mean(),white.call.std(), black.call.mean(), black.call.std()

(0.09650924056768417,
 0.2953455150127411,
 0.0644763857126236,
 0.24564945697784424)

In [44]:
np.sqrt(white.call.mean()*(1-white.call.mean())) #binomial standard deviation. Does it agree?

0.29528834560939421

In [45]:
np.sqrt(black.call.mean()*(1-black.call.mean())) #binomial standard deviation. Does it agree?

0.24559963639643398

In [50]:
diff = (white.call.mean() - black.call.mean()) 
se = np.sqrt(white.call.std()*white.call.std()+black.call.std()*black.call.std())/np.sqrt(len(white))
diff/se

4.114738324427746

This is a two sample hypothesis test comparing poopulation proportion (success probability follows binomial distribution).

Null hypothesis: There is no discrimination. Because the race label is applied randomly, we expect success rates of both "races" to be equal. So H0 : difference in success rate is zero.

In [41]:
stats.ttest_ind(white.call,black.call, 0) #two independent samples

Ttest_indResult(statistic=4.1147052908617514, pvalue=3.9408021031288859e-05)

p-value is very low. We reject the null hypothesis with 4 sigma statistical significance. There is discrimination.

In [52]:
diff, se

(0.03203285485506058, 0.0077849069198137949)

95% confidence interval

In [55]:
conf_int = stats.norm.interval(0.95, loc=diff, scale=se)
conf_int

(0.016774717669228893, 0.047290992040892266)

In [57]:
diff - conf_int[0], diff - conf_int[1]  

(0.015258137185831685, -0.015258137185831688)

In [58]:
(diff - conf_int[0])/diff #margin of error

0.47632773459844124

47% margin of error in difference.